In [1]:
import os
import pandas as pd
from yqcore import keyword_columns,readfile

In [2]:
filelist = ['data/' + i for i in os.listdir('data') if 'cibao' not in i]
filelist

['data/Lorna Jane .xlsx', 'data/Nike.xlsx', 'data/维多利亚的秘密 运动文胸.xlsx']

In [3]:
for i in filelist:
    filename = i[5:-4]
    
    rizhipath = 'res/' + filename + '【分析结果】.xlsx'
    pd.DataFrame([]).to_excel(rizhipath)
    excelWriter = pd.ExcelWriter(rizhipath)
    
    df = readfile(i,header=None)
    df.columns = ['【源数据】']
    df['【源数据】'] = df['【源数据】'].map(lambda x:str(x))
    df.to_excel(excelWriter,sheet_name='原始数据')
    
    cibao = readfile('data/cibao.csv')
    cibao[cibao.columns[-1]] = cibao[cibao.columns[-1]].map(lambda x:str(x))
    
    for x in cibao[cibao.columns[-1]]:
        df = keyword_columns(df,x)
    df.to_excel(excelWriter,sheet_name='词频检索')
    
    cdf = df.sum(axis=0)
    cibao['频数'] = cibao[cibao.columns[-1]].map(cdf)
    
    res = cibao.groupby(list(cibao.columns[:-2]))[['频数']].sum()
    res['占比'] = (res['频数']/res['频数'].sum()).map(lambda x: str(100*x)[:4] +'%')
    res = res.loc[res.频数!=0]
    res.sort_values(by='频数',ascending=False)
    res.to_excel(excelWriter,sheet_name='分组结果统计')
    
    res = cibao.groupby(list(cibao.columns[:-1]))[['频数']].sum()
    res['占比'] = (res['频数']/res['频数'].sum()).map(lambda x: str(100*x)[:4] +'%')
    res = res.loc[res.频数!=0]
    res.sort_values(by='频数',ascending=False)
    res.to_excel(excelWriter,sheet_name='索引展开统计')
    
    excelWriter.save()
    excelWriter.close()

## 设计
### 原数据 csv ，无head
### 词包 csv ，正负-一级-二级-检索，有head

In [4]:
print('Done')

Done
